# Fix RDRS PET units
We did the PET computations in `kg m-2 s-1`, but stored the `units` attribute as `mm hr-1`. Here we loop over all the RDRS files and correct this oversight.

In [21]:
import glob
import sys
import pandas as pd
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

from itertools import chain
from netCDF4 import Dataset

### Config handling

In [3]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [4]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name = cs.read_from_config(config_file,'cs_meta_name')
cs_unusable_name = cs.read_from_config(config_file,'cs_unusable_name')

# Basin folder
cs_basin_folder = cs.read_from_config(config_file, 'cs_basin_path')
basins_path = Path(data_path) / cs_basin_folder

### Data loading

In [5]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [6]:
# Open list of unusable stations; Enforce reading IDs as string to keep leading 0's
cs_unusable = pd.read_csv(cs_meta_path / cs_unusable_name, dtype={'Station_id': object})

### Loop over basins, find files, fix units

In [30]:
def netcdf_pet_units_mm_hr_to_kg_m2_s1(file, var_name='pet'):
    with Dataset(file, 'a') as nc_file:
        variable = nc_file.variables[var_name]
        if variable.units == 'mm hr**-1':
            variable.units = 'kg m**-2 s**-1'

In [32]:
for ix,row in cs_meta.iterrows():
    # DEBUGGING
    if ix != 1: continue

    # Get the paths
    basin_id, shp_lump_path, shp_dist_path, _, _ = cs.prepare_delineation_outputs(cs_meta, ix, basins_path)
    met_folder = basins_path / 'basin_data' / basin_id / 'forcing'

    # Find the RDRS files
    sub_folders = ['raw', 'lumped', 'distributed']
    rdrs_files = []
    for sub_folder in sub_folders:
        rdrs_files.append(glob.glob(str(met_folder / sub_folder / 'RDRS_*.nc')))

    # Flatten the list
    rdrs_files = list(chain.from_iterable(rdrs_files))
    rdrs_files.sort()
    
    # Fix the units
    print(f"updated PET units in:")
    for file in rdrs_files:
        netcdf_pet_units_mm_hr_to_kg_m2_s1(file)
        print(f' - {file}')

updated PET units in:
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_2009-06.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_2014-03.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_2010-09.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_1992-05.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_1989-02.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_1980-10.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_2008-10.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD003/forcing/raw/RDRS_1990-10.nc
 - /gpfs/tp/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data/CAN_01AD00